In [2]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
from config import weather_api_key
import requests
import pprint
from datetime import datetime
import time
from scipy.stats import linregress

In [3]:
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-180.000, 180.000, size=2000)
coordinates = list(zip(lats, lngs))

In [4]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

In [6]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [12]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
city_url = url + '&q=' + cities[0]
city_weather_single = requests.get(city_url).json()
city_weather_single
city_weather_single['weather'][0]['description']

'clear sky'

In [13]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    city_url = url + '&q=' + cities[i]

    print(f"Processing Record {record_count} of Set {set_count} | {city} | {i}")
    record_count += 1

    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    i += 1
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 14 of Set 16 | qaanaaq | 0
Processing Record 15 of Set 16 | nizhnevartovsk | 1
Processing Record 16 of Set 16 | vao | 2
Processing Record 17 of Set 16 | moussoro | 3
Processing Record 18 of Set 16 | port alfred | 4
Processing Record 19 of Set 16 | illoqqortoormiut | 5
City not found. Skipping...
Processing Record 20 of Set 16 | belushya guba | 6
City not found. Skipping...
Processing Record 21 of Set 16 | ponta do sol | 7
Processing Record 22 of Set 16 | mataura | 8
Processing Record 23 of Set 16 | yellowknife | 9
Processing Record 24 of Set 16 | rikitea | 10
Processing Record 25 of Set 16 | paamiut | 11
Processing Record 26 of Set 16 | goderich | 12
Processing Record 27 of Set 16 | hilo | 13
Processing Record 28 of Set 16 | provideniya | 14
Processing Record 29 of Set 16 | masuguru | 15
Processing Record 30 of Set 16 | nikolskoye | 16
Processing Record 31 of Set 16 | poum | 17
Processing Record 32 of Set 16 | hermanus | 18
Processing Record 33 of Set 16 | taolanaro |

In [18]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Qaanaaq,77.4840,-69.3632,5.59,72,8,8.41,GL,2021-11-02 04:57:00,NaN
1,Nizhnevartovsk,60.9344,76.5531,-0.38,85,40,4.47,RU,2021-11-02 04:57:01,NaN
2,Vao,-22.6667,167.4833,69.39,60,78,19.77,NC,2021-11-02 04:57:01,NaN
3,Moussoro,13.6408,16.4901,84.85,19,94,12.28,TD,2021-11-02 04:57:02,NaN
4,Port Alfred,-33.5906,26.8910,62.35,70,55,25.03,ZA,2021-11-02 04:57:02,NaN
...,...,...,...,...,...,...,...,...,...,...
1415,Namatanai,-3.6667,152.4333,84.09,72,84,3.67,PG,NaN,broken clouds
1416,San Rafael Del Sur,11.8485,-86.4384,75.42,86,97,4.65,NI,NaN,overcast clouds
1417,Sola,-13.8833,167.5500,80.64,75,81,24.29,VU,NaN,broken clouds
1418,Pictou,45.6834,-62.7153,49.44,49,0,18.41,CA,NaN,clear sky


In [19]:
new_order = ['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Current Description']
city_data_df = city_data_df[new_order]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Qaanaaq,GL,77.4840,-69.3632,5.59,72,8,8.41,NaN
1,Nizhnevartovsk,RU,60.9344,76.5531,-0.38,85,40,4.47,NaN
2,Vao,NC,-22.6667,167.4833,69.39,60,78,19.77,NaN
3,Moussoro,TD,13.6408,16.4901,84.85,19,94,12.28,NaN
4,Port Alfred,ZA,-33.5906,26.8910,62.35,70,55,25.03,NaN
...,...,...,...,...,...,...,...,...,...
1415,Namatanai,PG,-3.6667,152.4333,84.09,72,84,3.67,broken clouds
1416,San Rafael Del Sur,NI,11.8485,-86.4384,75.42,86,97,4.65,overcast clouds
1417,Sola,VU,-13.8833,167.5500,80.64,75,81,24.29,broken clouds
1418,Pictou,CA,45.6834,-62.7153,49.44,49,0,18.41,clear sky


In [20]:
city_data_df.to_csv('weather_data/WeatherPy_Database.csv')